<a href="https://colab.research.google.com/github/sugarforever/huggingface-tutorials/blob/main/hg_02_autoclass_chatglm_6b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 小白入门HuggingFace

## 02 Transformers的魔法 - AutoClass

`AutoClass` 是 `Transformers` 提供的，通过预训练模型的名称或路径自动检索其架构的接口。您只需要为任务选择合适的 `AutoClass`及其关联的预处理类，剩下的工作就交给 `AutoClass`。

### AutoTokenizer

`Tokenizer` 负责将文本预处理成数字数组，作为模型的输入。`Tokenization` 的过程受多条规则约束，包括如何划分单词，在何种粒度上划分单词等。

`AutoTokenizer` 提供了统一的接口，基于模型的名称，加载 `Tokenizer`。

注，需要用相同的模型名称实例化一个 `Tokenizer`，以确保您使用的是模型预训练时使用的相同 `Tokenization` 规则。

示例请见以下ChatGLT-6B模型的加载。

### AutoModel

与 `AutoTokenizer` 类似，`AutoModel` 提供了加载模型的统一接口。

## ChatGLM-6B

`ChatGLM-6B` 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。




## 代码实例

利用 `Transformers` 加载 `ChatGLM-6B` 实现中文问答。

In [1]:
!pip install git+https://github.com/huggingface/transformers sentencepiece --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00


### 从HuggingFace利用 `AutoClass` 加载 `ChatGLM-6B`


In [3]:
from transformers import AutoTokenizer, AutoModel

In [4]:
model_id = "THUDM/chatglm-6b"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



`Tokenizer` 返回一个包含以下信息:
- input_ids: 标记的数值表示。

In [8]:
encoding = tokenizer("你今天吃了吗？")
encoding

{'input_ids': [5, 98172, 66826, 63964, 31, 130001, 130004]}

### 利用 `AutoModel` 加载模型

- .half().cuda():
        .half(): 将模型的数据类型从float32改为float16，减小模型大小，加速计算。
        .cuda(): 将模型加载到GPU上，利用GPU进行加速。

- model.eval():
        将模型设置为eval模式,该模式下会关闭dropout等随机操作,可以改善推理效果。
        train模式下的BN统计数据计算是无意义的,eval模式用预统计的数据可以获得更准确的输出。

这些代码的目的：使模型在推理阶段获得更佳的性能，包括
1. 较快的速度
2. 更准确的输出

In [6]:
model = AutoModel.from_pretrained(model_id, trust_remote_code=True).half().cuda()

print(model.__class__)

model = model.eval()

prompt = "你好"
response, history = model.chat(tokenizer, prompt, history=[])

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

<class 'transformers_modules.THUDM.chatglm-6b.619e736c6d4cd139840579c5482063b75bed5666.modeling_chatglm.ChatGLMForConditionalGeneration'>


In [9]:
response

'你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。'

In [10]:
prompt = "什么是量子化？"
response, history = model.chat(tokenizer, prompt, history=[])
response

'量子化是一种数学方法，用于描述量子系统的性质和行为。在量子力学中，量子系统是由一系列微观粒子(称为“粒子”)组成的，这些粒子之间存在着特定的相互作用，使得量子系统具有一些与传统物理量不同的性质，例如量子叠加态和量子纠缠等。\n\n量子化方法可以用于描述量子系统的性质和行为，例如计算量子系统的态和相互作用，研究量子现象的机制，以及开发量子计算机等。在量子化学、量子信息学和量子物理学等领域，量子化方法都扮演着非常重要的角色。'